In [ ]:
import pandas as pd

df = pd.read_excel("2023 PR.xlsx")

In [ ]:
# Convert the 'Policy Issue Date' column to datetime format
df['Policy Issue date'] = pd.to_datetime(df['Policy Issue date'])

# Remove duplicates by keeping the latest 'Policy Issue Date' for each 'Initial Policy No'
df = df.sort_values(by='Policy Issue date', ascending=False).drop_duplicates(subset='Initial Policy No', keep='first')

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_PR dataset.csv', index=False)

In [ ]:
import pandas as pd

# Load the main and renewed data into DataFrames
base_data = pd.read_excel("Pvt Car Jan'23 to  Dec 23 base Final.xlsx")  
cleaned_pr_data = pd.read_excel("cleaned_PR dataset.xlsx")

In [ ]:
# Merge the datasets on 'Initial Policy No'
merged_data = base_data.merge(
    cleaned_pr_data[['Initial Policy No', 'Policy Start Date', 'Policy End Date', 'New Policy No',
                     'Total Premium Payable ', 'NCB % Previous Year', 'Previous Year NCB (Amount)', 'Age', 
                     'Nil Depreciation', 'Passenger Assist', 'Consumable Cover', 'Engine Safe Cover', 'Road Side Assistance', 
                     'Key Loss', 'Net TP Premium / War & SRCC','Net OD Premium', 'Detariff Disc Rate', 'No. of Claim Year 1', 'GST']],
    on='Initial Policy No',
    how='left'
)

# Rename the columns as specified
merged_data.rename(columns={
    'Policy Start Date': 'Next year Policy Start Date',
    'Policy End Date': 'Next year Policy End Date',
    'New Policy No': 'New Policy No',
    'Total Premium Payable ': 'Next year Total Premium Payable',
    'NCB % Previous Year': 'Next year NCB % Previous Year',
    'Net TP Premium / War & SRCC': 'Next year Total TP Premium',
    'Net OD Premium': 'Next year Net OD Premium',
    'Detariff Disc Rate': 'Next year Discount', 
    'No. of Claim Year 1': 'Next year claim', 
    'GST': 'Next year GST',
    'Age': 'Next year Age',
    'Nil Depreciation': 'Next year Nil Depreciation',
    'Passenger Assist': 'Next year Passenger Assist',
    'Consumable Cover': 'Next year Consumable Cover',
    'Engine Safe Cover': 'Next year Engine Safe Cover',
    'Road Side Assistance': 'Next year Road Side Assistance',
    'Key Loss': 'Next year Key Loss'
}, inplace=True)

# Save the merged data to a new CSV file
merged_data.to_csv("merged_base_data.csv", index=False)

In [7]:
import pandas as pd

base_data = pd.read_excel("Pvt Car Jan'23 to  Dec 23 base Final.xlsx")  
cleaned_pr_data = pd.read_excel("cleaned_PR dataset.xlsx")

In [11]:
# Add a 'Source' column to distinguish the origin of the data
base_data['Source'] = 'Base'
cleaned_pr_data['Source'] = 'Renewed'

# Remove duplicate columns from both DataFrames
base_data = base_data.loc[:, ~base_data.columns.duplicated()].copy()
cleaned_pr_data = cleaned_pr_data.loc[:, ~cleaned_pr_data.columns.duplicated()].copy()

# Reset the index for both DataFrames to avoid index conflicts
base_data.reset_index(drop=True, inplace=True)
cleaned_pr_data.reset_index(drop=True, inplace=True)

# Identify common columns between base_data and cleaned_pr_data
common_columns = list(set(base_data.columns) & set(cleaned_pr_data.columns))

# Filter cleaned_pr_data to include only the common columns
filtered_pr_data = cleaned_pr_data[common_columns + ['Source']]

# Add missing columns from base_data to filtered_pr_data with empty strings
for col in base_data.columns:
    if col not in filtered_pr_data.columns:
        filtered_pr_data[col] = ""

# Align the column order of filtered_pr_data with base_data
filtered_pr_data = filtered_pr_data[base_data.columns]

# Reset the index again to ensure no conflicts during concatenation
filtered_pr_data.reset_index(drop=True, inplace=True)
base_data.reset_index(drop=True, inplace=True)

# Concatenate the DataFrames row-wise
merged_data = pd.concat([base_data, filtered_pr_data], ignore_index=True)

# Replace NaN values with empty strings ("")
merged_data = merged_data.fillna("")

# Columns to duplicate based on 'Initial Policy No'
columns_to_duplicate = ['Insured Name', 'Renewal Type', 'Vehicle Segment', 'Decline', 
                        'New Branch Name 2', 'Zone 2', 'State2']

# Duplicate specified column details using forward and backward fill
for col in columns_to_duplicate:
    if col in merged_data.columns:
        merged_data[col] = merged_data.groupby('Initial Policy No')[col].transform(lambda x: x.ffill().bfill())

# Save the merged data to a new CSV file
merged_data.to_csv("Merged row (base&RH).csv", index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12512\268226419.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_pr_data[col] = ""
C:\Users\Admin\AppData\Local\Temp\ipykernel_12512\268226419.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_pr_data[col] = ""
C:\Users\Admin\AppData\Local\Temp\ipykernel_12512\268226419.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

InvalidIndexError: Reindexing only valid with uniquely valued Index objects